In [1]:
import pyodbc

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from pycaret.anomaly import *

In [2]:
## Conexion al DWH
cnxn = pyodbc.connect(
    driver='{SQL Server}',
    server='192.168.100.58',
    uid='bilectura',
    pwd='D1sp@p3l3s')
cursor = cnxn.cursor()
#cnxn.close()

In [3]:
fecha_inicio = "'2021-01-01'"
fecha_final = "'2023-01-24'"

In [9]:
codigo_SQL = f"SELECT A.VTAANO AS 'Ano', A.VTAMES AS 'Mes', A.DSTCOD AS 'Codigo distrito', A.VTAZNA AS 'Codigo zona', A.VTANIT AS 'Nit cliente', C.NITNOM AS 'Nombre cliente', A.VTAPRDCOD AS 'Codigo producto', B.TPOCOD AS 'Codigo tipo', B.LNACOD AS 'Codigo linea', B.GRPCOD AS 'Codigo grupo', B.CLDCOD AS 'Codigo calidad', SUM(A.VTAVLRVTA) AS 'Ventas', SUM(A.VTACST) AS 'Costo' FROM V_VTA_VTAHEC A LEFT JOIN DIM_PRODUCTOS B ON A.PRMCOD = B.PRMCOD AND A.VTAPRDCOD = B.PRDCOD LEFT JOIN DIM_NITS C ON A.VTANIT = C.NITNIT WHERE A.PRMCOD = 1 AND A.DSTCOD != 30 AND A.VTAZNA < 96 AND A.VTAFCH BETWEEN {fecha_inicio} AND {fecha_final} AND  B.TPOCOD NOT IN (990,999) GROUP BY A.VTAANO, A.VTAMES, A.DSTCOD, A.VTAZNA, A.VTANIT, C.NITNOM, A.VTAPRDCOD, B.TPOCOD, B.LNACOD, B.GRPCOD, B.CLDCOD"

In [10]:
#Cargue de la data desde el ERP de Dispapeles y se guarda en df
cursor.execute(codigo_SQL)
rows = cursor.fetchall()
df = pd.DataFrame.from_records(rows, columns=[col[0] for col in cursor.description])

In [7]:
maestra_gerencias = pd.read_csv('C:/Users/tcardenas/OneDrive/OneDrive - Grupo DISPAPELES/Documents/ML-Dispapeles-TomasCaLo/Anomaly detection/Maestra_gerencias.csv', delimiter= ";")
maestra_zonas = pd.read_csv('C:/Users/tcardenas/OneDrive/OneDrive - Grupo DISPAPELES/Documents/Análisis clustering/Maestra_zonas.csv', delimiter= ";")
maestra_zonas.rename(columns= {"Codigo zona actual": "Codigo zona"})

,Codigo distrito,Codigo zona,Nombre tipo zona
0,10,1,Artes graficas
1,10,2,Artes graficas
2,10,4,Artes graficas
3,10,5,Artes graficas
4,10,7,Artes graficas
5,10,9,Artes graficas
6,10,10,TIG
7,10,11,TIG
8,10,12,Sector oficial
9,10,13,TIG


In [22]:
df_merged = pd.merge(df, maestra_gerencias, on= "Codigo producto")
df_merged["Margen porcentaje"] = round((1 - (df_merged["Costo"].astype(int)) / (df_merged["Ventas"].astype(int))).astype(float) * 100,2)

In [23]:
df_merged

,Ano,Mes,Codigo distrito,Codigo zona,Nit cliente,Nombre cliente,Codigo producto,Codigo tipo,Codigo linea,Codigo grupo,Codigo calidad,Ventas,Costo,Gerente,Linea CAM,Margen porcentaje
0,2021,3,25,83,814000416,EM.TECNOLOGIA IMPRENTA Y COMUNI.DE NARIÑ,930,101,1,1,48,1125468.8000,756022,Papel para escritura,Bond,32.83
1,2021,1,71,64,800219192,CENTRO MEDICO DE ESPECIALISTAS CME S.A,930,101,1,1,48,103270.5000,74785,Papel para escritura,Bond,27.58
2,2021,8,20,11,890327720,PAPELERIA FERNANDEZ S.A.S,930,101,1,1,48,215982.0000,153490,Papel para escritura,Bond,28.93
3,2021,11,90,20,809012539,LEON GRAFICAS S.A.S.,930,101,1,1,48,1397280.0000,830473,Papel para escritura,Bond,40.57
4,2021,4,70,16,7527272,OROZCO LOPEZ ERNEY,930,101,1,1,48,473900.0000,372513,Papel para escritura,Bond,21.39
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
524041,2022,5,20,63,900166251,HEALTHY AMERICA COLOMBIA S.A.S,22907,235,103,112,326,26666.6500,23068,Elementos de Proteccion Personal,Proteccion personal,13.49
524042,2021,2,10,7,830047852,PUNTO VISUAL PUBLICIDAD LTDA,7017,180,166,242,534,3808232.0000,2560778,Comunicacion visual,Sustratos de linea,32.76
524043,2022,12,10,71,800036578,FUNDACION NIÑOS DE LOS ANDES,55551,235,17,136,401,598654.0000,598621,Clean it,Clean it,0.01
524044,2021,3,10,7,800234331,GRAFIQ EDITORES S.A.S.,27659,180,196,205,25,357500.0000,201126,Comunicacion visual,Otros,43.74


In [ ]:
df_vta = df_merged.groupby(["Ano", "Mes", "Codigo distrito", "Codigo zona", "Nit cliente", "Nombre cliente", "Codigo tipo",
                            "Codigo linea", "Codigo grupo", "Codigo calidad", "Gerente", "Línea CAM"]).agg({"Ventas": np.sum,
                                                                                                            "Costo": np.sum})
df_vta = df_vta.reset_index(col_level= 1)
df_vta["Codigo distrito-zona"] = df_vta["Codigo distrito"].astype(str) + "-" + df_vta["Codigo zona"].astype(str)
df_vta["Margen"] = df_vta["Ventas"] - df_vta["Costo"]
df_vta["Ventas"] = df_vta["Ventas"].astype(int)
df_vta["Margen"] = df_vta["Margen"].astype(int)

In [ ]:
df_vta.head()

In [ ]:
df_vta.corr()*100

In [ ]:
corr_matrix = df_vta.corr()
mask = np.zeros_like(corr_matrix)
mask[np.triu_indices_from(mask)] = True
sns.heatmap(corr_matrix, mask= mask, square= True)

In [ ]:
list_dst_znas = (df_vta["Codigo distrito"].astype(str) + "-" + df_vta["Codigo zona"].astype(str)).unique()
categorical_columns = ["Ano", "Mes", "Codigo distrito", "Codigo zona", "Nit cliente","Nombre cliente", "Codigo tipo",
                        "Codigo linea", "Codigo grupo", "Codigo calidad", "Gerente", "Línea CAM", "Codigo distrito-zona"]
ignore_columns = ["Ano", "Mes", "Codigo grupo", "Codigo calidad", "Codigo distrito-zona"]
numerical_columns = ["Ventas", "Costo", "Margen"]

In [ ]:
models()

In [ ]:
consolidado = pd.DataFrame([])
for n, dtozna in enumerate(list_dst_znas):
    print(f"Procesando {n+1} de {len(list_dst_znas)+1}. {(n+1) / (len(list_dst_znas)+1):.1%}")
    #Creacion del subdataset
    df_dto = df_vta[df_vta["Codigo distrito-zona"] == dtozna]
    
    #Setup modelo
    exp_ano101 = setup(df_dto, 
                        session_id = 42,
                        silent=True,
                        categorical_features = categorical_columns,
                        numeric_features = numerical_columns,
                        ignore_features = ignore_columns,
                        normalize = True,
                        normalize_method = 'robust',
                        verbose = False)
    
    #Modelo iforest
    iforest = create_model('iforest')
    iforest_results = assign_model(iforest)
    resultado_iforest = iforest_results.loc[(iforest_results["Anomaly"] == 1) & 
                                            (iforest_results["Anomaly_Score"] >= 0.02) &
                                            ((iforest_results["Ventas"] >= 2000000) | (iforest_results["Margen"] < 0)) &
                                            (iforest_results["Ano"] == 2023) & 
                                            (iforest_results["Mes"] == 1)
                                            ]

    #Append a una base general
    consolidado = consolidado.append(resultado_iforest)

#Descarga de la base general
consolidado.index = pd.RangeIndex(len(consolidado.index))
consolidado.to_csv(f"C:/Users/tcardenas/OneDrive/OneDrive - Grupo DISPAPELES/Documents/ML-Dispapeles-TomasCaLo/Anomaly detection/Anomalies_iforest/Anomalies ene22 25-1-23.csv",
                    encoding= 'utf-8', index= False, sep= ";", decimal = ",")

print("Proceso completo 100%")